In [30]:
import torch
import numpy as np

# Feature (temp, rain, hum)
features = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# # GT (apples, oranges)
# gt = np.array([[56, 70], 
#                     [81, 101], 
#                     [119, 133], 
#                     [22, 37], 
#                     [103, 119],
#                     [57, 69], 
#                     [80, 102], 
#                     [118, 132], 
#                     [21, 38], 
#                     [104, 118], 
#                     [57, 69], 
#                     [82, 100], 
#                     [118, 134], 
#                     [20, 38], 
#                     [102, 120]], 
#                    dtype='float32')

gt = np.array([[1], 
                    [2], 
                    [1], 
                    [2], 
                    [1],
                    [2], 
                    [2], 
                    [2], 
                    [1], 
                    [1], 
                    [1], 
                    [1], 
                    [1], 
                    [2], 
                    [2]], 
                   dtype='float32')

In [31]:
feature = torch.from_numpy(features)
gt_value = torch.from_numpy(gt)

**Dataset and DataLoader :**  **Dataset** - helps to work with rows from feature and gt as tuples. **Dataloader** provides with batches whcih help us to split our data and work easly.

In [32]:
from torch.utils.data import TensorDataset, DataLoader

# Define the dataset

train_ds = TensorDataset(feature, gt_value)
train_ds[0:3] # goes to gt and feature data and get 3 rows of it 

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[1.],
         [2.],
         [1.]]))

In [33]:
# define batches and use DataLoader

BATCH_SIZE = 5 
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True)

In [34]:
for x, y in train_dl :
  print(x)
  print(y) 
  break # if I don't use break my for loop gets all the dl batch size (5 sets). But in this case, it takes only 2 sets

tensor([[91., 87., 65.],
        [68., 96., 71.],
        [69., 96., 70.],
        [74., 66., 43.],
        [91., 88., 64.]])
tensor([[2.],
        [1.],
        [1.],
        [2.],
        [2.]])


**nn.Linear** : instead of initializing the WEIGHTS and BIASES manually, we can use nn.Linear module which already initialized for us.

In [41]:
import torch.nn as nn
my_model = nn.Sequential(nn.Linear(3,5), nn.Linear(5,3), nn.Linear(3,1)) # means 3 features amd 2 gt 
# print(my_model.weight)
# print(my_model.bias)

In [42]:
# make a prediction

preds = my_model(feature)
preds

tensor([[ 0.6872],
        [ 1.3402],
        [-2.8276],
        [ 3.2949],
        [ 0.2589],
        [ 0.8057],
        [ 1.4911],
        [-2.7131],
        [ 3.1765],
        [ 0.2914],
        [ 0.8381],
        [ 1.4587],
        [-2.9784],
        [ 3.2625],
        [ 0.1405]], grad_fn=<AddmmBackward0>)

**Loss Function** : Alread has build-in form

In [43]:
import torch.nn.functional as F

my_loss_function = F.mse_loss # mse_loss is MeanSquareError build in function as mse_loss

loss = my_loss_function(my_model(feature),gt_value ) # just apply my model to the feature not to the gt
print(loss)

tensor(4.5109, grad_fn=<MseLossBackward0>)


If i have certain question about something like models : just use : **?nn.Linear** which gives the all info. about the model.

**OPTIMIZATION :** Instead of using grad. as I did without build-in functions, I can use the build in function **SGD (stochastic gradient descent) **

In [44]:
# defining my optimizer

my_optimizer = torch.optim.SGD(my_model.parameters(), lr = 0.001)

### **Train the Model**

In [45]:
def  my_train_model(num_epoch, my_optimizer, loss, my_model):
    for epoch in range (num_epoch):
        for x , y in train_dl:
            pred = my_model(x) # generate a prediction
            loss = my_loss_function(pred, y) # calculate my loss value
            my_optimizer.zero_grad() # reset the derivatives to 0
            loss.backward() # compute derivatives
            my_optimizer.step() # update the parameters by uing the derivatives 
            
            # process
        if (epoch + 1 ) % 10 == 0:
            print('Epoch[{}/{}], Loss: {:.4f}'.format(epoch+1, num_epoch, loss.item()))


In [46]:
my_train_model(100, my_optimizer,loss,my_model) # I don't have to forget the order of parms. when i am passing them

Epoch[10/100], Loss: 0.4126
Epoch[20/100], Loss: 0.3545
Epoch[30/100], Loss: 0.1584
Epoch[40/100], Loss: 0.1504
Epoch[50/100], Loss: 0.1889
Epoch[60/100], Loss: 0.2818
Epoch[70/100], Loss: 0.1078
Epoch[80/100], Loss: 0.2850
Epoch[90/100], Loss: 0.3668
Epoch[100/100], Loss: 0.3780


In [13]:
preds = my_model(feature)
preds

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward0>)